In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import sys
# Add current working directory to path for module imports
sys.path.append(os.getcwd())
# Show which WIX data item ID is being used
print("WIX_DAILY_BUDGET_DATA_ITEM_ID =", os.getenv("WIX_DAILY_BUDGET_DATA_ITEM_ID"))


WIX_DAILY_BUDGET_DATA_ITEM_ID = 8f1c08f2-5ff8-4c40-8aee-4f5519e6b8a1


In [2]:
# Instantiate WixClient with explicit credentials
from wix_client import WixClient
client = WixClient(
    api_key=os.getenv("WIX_API_KEY"),
    daily_budget_item_id=os.getenv("WIX_DAILY_BUDGET_DATA_ITEM_ID"),
    account_id=os.getenv("WIX_ACCOUNT_ID"),
    site_id=os.getenv("WIX_SITE_ID")
)
print("Budget via WixClient:", client.get_daily_budget())

# Test fetching daily budget from WIX
from wix_client import get_daily_budget
budget = get_daily_budget()
print(f"Daily budget from WIX: {budget}")

Budget via WixClient: 0.798699
Daily budget from WIX: 0.798699


In [3]:
# Test fetching order book from LATOKEN
from latoken_client import get_order_book
order_book = get_order_book(limit=10)
print("Top 5 bids:")
for bid in order_book.get("bids", [])[:5]:
    print(bid)
print("Top 5 asks:")
for ask in order_book.get("asks", [])[:5]:
    print(ask)

Top 5 bids:
{'price': '0.00910000000', 'quantity': '175.62', 'cost': '1.59814200000', 'accumulated': '1.59814200000'}
{'price': '0.00850299999', 'quantity': '351.09', 'cost': '2.9853182664891', 'accumulated': '4.5834602664891'}
{'price': '0.00850200000', 'quantity': '25.00', 'cost': '0.21255000000', 'accumulated': '4.7960102664891'}
{'price': '0.00811000000', 'quantity': '25.00', 'cost': '0.20275000000', 'accumulated': '4.9987602664891'}
{'price': '0.00230100009', 'quantity': '480.52', 'cost': '1.1056765632468', 'accumulated': '6.1044368297359'}
Top 5 asks:
{'price': '0.03299000000', 'quantity': '30.06', 'cost': '0.99167940000', 'accumulated': '0.99167940000'}
{'price': '0.03300000000', 'quantity': '292.32', 'cost': '9.64656000000', 'accumulated': '10.63823940000'}
{'price': '0.03899900000', 'quantity': '20.84', 'cost': '0.81273916000', 'accumulated': '11.45097856000'}
{'price': '0.04700000000', 'quantity': '138.52', 'cost': '6.51044000000', 'accumulated': '17.96141856000'}
{'price': '

In [4]:
from wix_client import get_daily_budget
from latoken_client import LatokenClient

# Determine how many TDG to purchase today based on the daily budget and order book
budget = get_daily_budget()  # USD
client = LatokenClient()
result = client.calculate_purchase_amount(budget, limit=50)

# Only print when quantity is positive
qty = result['total_quantity']
if qty <= 0:
    print('No TDG to purchase today (budget too low or no asks).')
else:
    print(f'Daily budget (USD): {budget:.6f}')
    print(f'Total TDG purchasable: {qty:.4f} TDG')
    print(f"Total cost (USD): {result['total_cost']:.4f} USD")
    print(f"Average price (USD/TDG): {result['average_price']:.6f}")
    print('Purchase breakdown per ask level:')
    for entry in result['purchases']:
        print(f" - {entry['quantity']:.4f} TDG @ {entry['price']:.8f} USD = {entry['cost']:.4f} USD")
